# Creating FABnet IPv4 Network

FABRIC provides a pair of layer 3 IP networking services across every FABRIC site (FABnetv4 and FABnetv6). You can think of this service as a private internet that connects experiments across the testbed using FABRIC's high-performance network links. 

This notebook describes how to use the FABnetv4 service which is FABRIC's private IPv4 internet.   

## Step 1:  Configure the Environment


In [153]:
import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

os.environ['FABRIC_PROJECT_ID'] = '1630021f-0a0c-4792-a241-997f410d36e1'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']="minawm_0041350787"
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/Mina_Bastion_Key2'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

## Step 2: Import the FABLlib Library


In [154]:
import json
import traceback

from fabrictestbed_extensions.fablib.fablib import fablib


## Step 3 (Optional): Query for Available Tesbed Resources and Settings

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity.

In [155]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

Name      CPUs  Cores    RAM (G)    Disk (G)       Basic (100 Gbps NIC)    ConnectX-6 (100 Gbps x2 NIC)    ConnectX-5 (25 Gbps x2 NIC)    P4510 (NVMe 1TB)    Tesla T4 (GPU)    RTX6000 (GPU)
------  ------  -------  ---------  -------------  ----------------------  ------------------------------  -----------------------------  ------------------  ----------------  ---------------
DALL         6  192/192  1536/1536  60600/60600    381/381                 2/2                             2/2                            10/10               2/2               3/3
WASH         6  192/192  1536/1536  60600/60600    381/381                 2/2                             2/2                            10/10               2/2               3/3
MASS         4  128/128  1024/1024  55800/55800    254/254                 2/2                             0/0                            6/6                 0/0               3/3
MAX         10  302/320  2494/2560  116280/116400  624/635                 0/2

## Step 4: Create the Experiment Slice

The following creates two nodes, on different sites, with basic NICs connected to FABRIC's FABnetv4 internet.  

Two nodes are created and one NIC component is added to each node.  This example uses components of model `NIC_Basic` which are SR-IOV Virtual Function on a 100 Gpbs Mellanox ConnectX-6 PCI device. The VF is accessed by the node via PCI passthrough. Other NIC models are listed below. When using dedicated PCI devices the whole physical device is allocated to one node and the device is accessed by the node using PCI passthrough. Calling the `get_interfaces()` method on a component will return a list of interfaces. Many dedicated NIC components may have more than one port.  Either port can be connected to the network.

Next, add a separate `l3network` for each site and pass the list of interfaces on that site that you want to connect to FABnetv4. All interfaces passed to `l3network` must be on the same site and each network will be placed on that site.  By default, a node is put on a random site.  If you want to ensure that your nodes are all on different sites you can specify the name of the sites in the `add_node` methode.  You can use the `fablib.get_random_site()` method to get a set of random site names that guarantee that the sites are different. 


NIC component models options:
- NIC_Basic: 100 Gbps Mellanox ConnectX-6 SR-IOV VF (1 Port)
- NIC_ConnectX_5: 25 Gbps Dedicated Mellanox ConnectX-5 PCI Device (2 Ports) 
- NIC_ConnectX_6: 100 Gbps Dedicated Mellanox ConnectX-6 PCI Device (2 Ports) 

In [156]:
slice_name = 'MySlice1'
# [site1,site2] = fablib.get_random_sites(count=2)
# print(f"Sites: {site1}, {site2}")
site1 = 'MAX'
site2 = 'WASH'

node1_name = 'Node1'
node2_name = 'Node2'

network1_name='net1'
network2_name='net2'

node1_nic_name = 'nic1'
node2_nic_name = 'nic2'

In [157]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    # Node1
    node1 = slice.add_node(name=node1_name, site=site1, cores=32, # cores=64,
                           image='default_ubuntu_20')
    iface1 = node1.add_component(model='NIC_ConnectX_5', name=node1_nic_name).get_interfaces()[0]
    
    # Node2
    node2 = slice.add_node(name=node2_name, site=site2, cores=32,# cores=64,
                           image='default_ubuntu_20')
    iface2  = node2.add_component(model='NIC_ConnectX_5', name=node2_nic_name).get_interfaces()[0]
    
    # NetworkS
    net1 = slice.add_l3network(name=network1_name, interfaces=[iface1], type='IPv4')
    net2 = slice.add_l3network(name=network2_name, interfaces=[iface2], type='IPv4')
    
    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")


-----------  ------------------------------------
Slice Name   MySlice1
Slice ID     a0d39ce3-9b79-4e20-bdc5-ffeba6904f4f
Slice State  StableOK
Lease End    2022-06-25 15:56:07 +0000
-----------  ------------------------------------

Retry: 14, Time: 158 sec

ID                                    Name    Site    Host                          Cores    RAM    Disk  Image              Management IP                           State    Error
------------------------------------  ------  ------  --------------------------  -------  -----  ------  -----------------  --------------------------------------  -------  -------
75624438-0056-4f2d-b98a-66ff3c0bf1ac  Node1   MAX     max-w5.fabric-testbed.net        32    128      10  default_ubuntu_20  63.239.135.120                          Active
5f1d73ed-f9db-4b84-9390-2acc8f45fe0e  Node2   WASH    wash-w3.fabric-testbed.net       32    128      10  default_ubuntu_20  2001:400:a100:3020:f816:3eff:fe48:5a2c  Active

Time to stable 158 seconds
Runni

## Step 5: Observe the Slice's Attributes

### Print the slice

In [158]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

-----------  ------------------------------------
Slice Name   MySlice1
Slice ID     a0d39ce3-9b79-4e20-bdc5-ffeba6904f4f
Slice State  StableOK
Lease End    2022-06-25 15:56:07 +0000
-----------  ------------------------------------


## Print the Node List

In [159]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice.list_nodes()}")
except Exception as e:
    print(f"Exception: {e}")

ID                                    Name    Site    Host                          Cores    RAM    Disk  Image              Management IP                           State    Error
------------------------------------  ------  ------  --------------------------  -------  -----  ------  -----------------  --------------------------------------  -------  -------
75624438-0056-4f2d-b98a-66ff3c0bf1ac  Node1   MAX     max-w5.fabric-testbed.net        32    128      10  default_ubuntu_20  63.239.135.120                          Active
5f1d73ed-f9db-4b84-9390-2acc8f45fe0e  Node2   WASH    wash-w3.fabric-testbed.net       32    128      10  default_ubuntu_20  2001:400:a100:3020:f816:3eff:fe48:5a2c  Active


## Print the Node Details

In [160]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

-----------------  -------------------------------------------------------------------------------------------------------
ID                 75624438-0056-4f2d-b98a-66ff3c0bf1ac
Name               Node1
Cores              32
RAM                128
Disk               10
Image              default_ubuntu_20
Image Type         qcow2
Host               max-w5.fabric-testbed.net
Site               MAX
Management IP      63.239.135.120
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/.ssh/id_rsa -J minawm_0041350787@bastion-1.fabric-testbed.net ubuntu@63.239.135.120
-----------------  -------------------------------------------------------------------------------------------------------
-----------------  -------------------------------------------------------------------------------------------------------------------------------
ID                 5f1d73ed-f9db-4b84-9390-2acc8f45fe0e
Name               Node2
Cores              32
RAM                128
Disk  

## Print the Interfaces

In [161]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice.list_interfaces()}")
except Exception as e:
    print(f"Exception: {e}")

Name           Node    Network      Bandwidth  VLAN    MAC                Physical OS Interface    OS Interface
-------------  ------  ---------  -----------  ------  -----------------  -----------------------  --------------
Node1-nic1-p1  Node1   net1                25          04:3F:72:FA:77:FC  ens7                     ens7
Node1-nic1-p2  Node1                       25          04:3F:72:FA:77:FD  ens8                     ens8
Node2-nic2-p1  Node2   net2                25          B8:CE:F6:3A:63:12  ens7                     ens7
Node2-nic2-p2  Node2                       25          B8:CE:F6:3A:63:13  ens8                     ens8


## Step 6: Configure IP Addresses

Some experiments use FABRIC layer 2 networks to enable deploying non-IP layer 3 networks.  If this describes your experiment, your nodes and network are ready. You can now login to the nodes and deploy your experiment.

Most users will want to configure IP addresses on there new nodes.  FABlib provides some useful methods to help you configure basic IP addresses. 

### Get the Assigned Subnet

FABnetv4 networks are assigned a subnet and gateway by FABRIC.  You can get the subnet and available IPs from the FABlib objects. 

In [162]:
try:
    network1 = slice.get_network(name=network1_name)
    network1_available_ips = network1.get_available_ips()
    print(f"{network1}")
    
    network2 = slice.get_network(name=network2_name)
    network2_available_ips =  network2.get_available_ips()
    print(f"{network2}")
except Exception as e:
    print(f"Exception: {e}")

-----------------  ------------------------------------
ID                 7dd894dc-8e17-40a6-9c7c-9fc33f6a5cfe
Name               net1
Layer              L3
Type               FABNetv4
Site               MAX
Gateway            10.130.1.1
L3 Subnet          10.130.1.0/24
Reservation State  Active
Error Message
-----------------  ------------------------------------
-----------------  ------------------------------------
ID                 f8c26ea4-ba34-4881-871e-db0c9a0da4ab
Name               net2
Layer              L3
Type               FABNetv4
Site               WASH
Gateway            10.133.1.1
L3 Subnet          10.133.1.0/24
Reservation State  Active
Error Message
-----------------  ------------------------------------


### Configure Node1

Get the node and the interface you wish to configure.  You can use `node.get_interface` to get the interface that is connected to the specified network.  Then `pop` an IP address from the list of available IPs and call `iface.ip_addr_add` to set the IP and subnet.  

Then set a route from *this network* to the *other network* through the specified gateway.


Optionally, use the `node.execute()` method to show the results of adding the IP address and route.



In [163]:
try:
    target_ip = network1_available_ips.pop(0)
    node1 = slice.get_node(name=node1_name)        
    node1_iface = node1.get_interface(network_name=network1_name)  
    node1_iface.ip_addr_add(addr=target_ip, subnet=network1.get_subnet())
    
    node1.ip_route_add(subnet=network2.get_subnet(), gateway=network1.get_gateway())
    
    stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')
    print (stdout)
    
    stdout, stderr = node1.execute(f'ip route list')
    print (stdout)
except Exception as e:
    print(f"Exception: {e}")

3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 04:3f:72:fa:77:fc brd ff:ff:ff:ff:ff:ff
    inet 10.130.1.2/24 scope global ens7
       valid_lft forever preferred_lft forever
    inet6 fe80::63f:72ff:fefa:77fc/64 scope link 
       valid_lft forever preferred_lft forever

default via 10.20.4.1 dev ens3 proto dhcp src 10.20.4.203 metric 100 
10.20.4.0/23 dev ens3 proto kernel scope link src 10.20.4.203 
10.130.1.0/24 dev ens7 proto kernel scope link src 10.130.1.2 
10.133.1.0/24 via 10.130.1.1 dev ens7 
169.254.169.254 via 10.20.4.1 dev ens3 proto dhcp src 10.20.4.203 metric 100 



### Configure Node2

Repeat the steps to add the next available IP to the second node and a route to the first network.

In [164]:
try:
    node2 = slice.get_node(name=node2_name)        
    node2_iface = node2.get_interface(network_name=network2_name)  
    node2_iface.ip_addr_add(addr=network2_available_ips.pop(0), subnet=network2.get_subnet())
    
    node2.ip_route_add(subnet=network1.get_subnet(), gateway=network2.get_gateway())
    
    stdout, stderr = node2.execute(f'ip addr show {node2_iface.get_os_interface()}')
    print (stdout)
    
    stdout, stderr = node2.execute(f'ip route list')
    print (stdout)
except Exception as e:
    print(f"Exception: {e}")

3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether b8:ce:f6:3a:63:12 brd ff:ff:ff:ff:ff:ff
    inet 10.133.1.2/24 scope global ens7
       valid_lft forever preferred_lft forever
    inet6 fe80::bace:f6ff:fe3a:6312/64 scope link 
       valid_lft forever preferred_lft forever

default via 10.20.4.1 dev ens3 proto dhcp src 10.20.4.181 metric 100 
10.20.4.0/23 dev ens3 proto kernel scope link src 10.20.4.181 
10.130.1.0/24 via 10.133.1.1 dev ens7 
10.133.1.0/24 dev ens7 proto kernel scope link src 10.133.1.2 
169.254.169.254 via 10.20.4.11 dev ens3 proto dhcp src 10.20.4.181 metric 100 



In [165]:
print(node1.execute('sudo apt update && sudo apt install -y iperf iperf3'))

("Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease\nGet:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]\nGet:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]\nGet:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]\nGet:5 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [1583 kB]\nGet:6 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 Packages [8628 kB]\nGet:7 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe Translation-en [5124 kB]\nGet:8 http://security.ubuntu.com/ubuntu focal-security/main Translation-en [268 kB]\nGet:9 http://security.ubuntu.com/ubuntu focal-security/main amd64 c-n-f Metadata [10.6 kB]\nGet:10 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [1036 kB]\nGet:11 http://security.ubuntu.com/ubuntu focal-security/restricted Translation-en [147 kB]\nGet:12 http://security.ubuntu.com/ubuntu focal-secu

In [166]:
print(node2.execute('sudo apt update && sudo apt install -y iperf iperf3'))

("Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease\nGet:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]\nGet:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]\nGet:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]\nGet:5 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 Packages [8628 kB]\nGet:6 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe Translation-en [5124 kB]\nGet:7 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 c-n-f Metadata [265 kB]\nGet:8 http://nova.clouds.archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [144 kB]\nGet:9 http://nova.clouds.archive.ubuntu.com/ubuntu focal/multiverse Translation-en [104 kB]\nGet:10 http://nova.clouds.archive.ubuntu.com/ubuntu focal/multiverse amd64 c-n-f Metadata [9136 B]\nGet:11 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [1935 kB]\nGet:12 http://secur

---

In [207]:
print(node1.upload_file('command1.sh', 'command1.sh'))

-rw-rw-r--   1 1000     1000          974 24 Jun 16:52 ?


In [208]:
print(node2.upload_file('command1.sh', 'command1.sh'))

-rw-rw-r--   1 1000     1000          974 24 Jun 16:52 ?


In [209]:
print(node1.execute('chmod +x command1.sh'))

('', '')


In [210]:
print(node2.execute('chmod +x command1.sh'))

('', '')


In [211]:
print(node1.execute('./command1.sh'))

('net.ipv4.tcp_congestion_control = cubic\nnet.core.rmem_max = 2147483647\nnet.core.wmem_max = 2147483647\n', 'sysctl: setting key "net.ipv4.tcp_allowed_congestion_control": No such file or directory\nsysctl: setting key "net.ipv4.tcp_available_congestion_control"\n')


In [212]:
print(node2.execute('./command1.sh'))

('net.ipv4.tcp_congestion_control = cubic\nnet.core.rmem_max = 2147483647\nnet.core.wmem_max = 2147483647\n', 'sysctl: setting key "net.ipv4.tcp_allowed_congestion_control": No such file or directory\nsysctl: setting key "net.ipv4.tcp_available_congestion_control"\n')


---

In [192]:
print(node1.execute('iperf3 -s > /dev/null 2>&1 &'))

('', '')


In [219]:
import ipywidgets, IPython

import matplotlib.pyplot as plt

p_widget = ipywidgets.IntSlider(min=1, max=64, step=1)
w_widget = ipywidgets.IntSlider(min=32, max=1024, step=32)
r_widget = ipywidgets.Checkbox(description="reverse")
u_widget = ipywidgets.RadioButtons(options=['k', 'M'], description='unit for "-w"')
button = ipywidgets.Button(description="Run and plot!")
reset_graph = ipywidgets.Button(description="Reset Graph!")
ui = ipywidgets.VBox([p_widget, w_widget, u_widget, r_widget, button, reset_graph])

IPython.display.display(ui)

output = ipywidgets.Output()


xdata_for = []
ydata_for = []
zdata_for = []
xdata_back = []
ydata_back = []
zdata_back = []

xdata_for_reverse = []
ydata_for_reverse = []
zdata_for_reverse = []
xdata_back_reverse = []
ydata_back_reverse = []
zdata_back_reverse = []

@output.capture()
def on_reset_clicked(b):
#     print(xdata_for)
    xdata_for.clear()
    ydata_for.clear()
    zdata_for.clear()
    xdata_back.clear()
    ydata_back.clear()
    zdata_back.clear()

    xdata_for_reverse.clear()
    ydata_for_reverse.clear()
    zdata_for_reverse.clear()
    xdata_back_reverse.clear()
    ydata_back_reverse.clear()
    zdata_back_reverse.clear()
    
    plt.figure(figsize=(15, 15))
    
    ax = plt.axes(projection='3d')
    
    ax.axes.set_xlim3d(left=0, right=max([10, max(xdata_for, default=0), max(xdata_back, default=0), max(xdata_for_reverse, default=0), max(xdata_back_reverse, default=0)]))
    ax.axes.set_ylim3d(bottom=0, top=max([1, max(ydata_for, default=0), max(ydata_back, default=0), max(ydata_for_reverse, default=0), max(ydata_back_reverse, default=0)]))
    ax.axes.set_zlim3d(bottom=0, top=max([10, max(zdata_for, default=0), max(zdata_back, default=0), max(zdata_for_reverse, default=0), max(zdata_back_reverse, default=0)]))
    
    ax.plot3D(xdata_for_reverse, ydata_for_reverse, zdata_for_reverse, marker='P', markersize=12)
    ax.plot3D(xdata_back_reverse, ydata_back_reverse, zdata_back_reverse, marker='+', markersize=12)
    ax.plot3D(xdata_for, ydata_for, zdata_for, marker='x', markersize=12)
    ax.plot3D(xdata_back, ydata_back, zdata_back, marker='X', markersize=12)
    
    IPython.display.clear_output(wait=True)
    plt.show()

@output.capture()
def on_button_clicked(b):
    
    p_widget.disabled = True
    w_widget.disabled = True
    r_widget.disabled = True
    button.disabled = True
    reset_graph.disabled = True

    plt.figure(figsize=(15, 15))
    
    ax = plt.axes(projection='3d')
    
    reverse = ''
    if(ui.children[2].value):
        reverse = ' -R'
    
    command_string = 'iperf3 -c ' + str(target_ip) + ' -P ' + str(ui.children[0].value) + ' -w ' + str(ui.children[1].value) + ui.children[2].value + reverse
    
    iperf_string, err = node2.execute(command_string)
    
    IPython.display.clear_output(wait=True)
    
    p_widget.disabled = False
    w_widget.disabled = False
    r_widget.disabled = False
    button.disabled = False
    reset_graph.disabled = False

    print(command_string)
    
    if(err != ''):
        print(err)
    else:
        iperf_strings = iperf_string.splitlines()
        if(len(iperf_strings) > 3):
            output1 = iperf_string.splitlines()[-4]
            index = output1.index('bits/sec') - 6
            unit = output1[output1.index('bits/sec') - 1: output1.index('bits/sec')]
            output1 = float(output1[index:index+4])
            if(unit == 'M'):
                output1 /= 1000
            print("sender: " + str(output1))
    #         output1 = output1[:1]
            output2 = iperf_string.splitlines()[-3]
            index = output2.index('bits/sec') - 6
            unit = output2[output2.index('bits/sec') - 1: output2.index('bits/sec')]
            output2 = float(output2[index:index+4])
            if(unit == 'M'):
                output2 /= 1000
            print("receiver: " + str(output2))
    #         output2 = output2[:1]

        y_value = ui.children[1].value
        if(ui.children[2].value == 'k'):
            y_value /= 1000
        if(reverse):
            xdata_for_reverse.append(ui.children[0].value)
            ydata_for_reverse.append(y_value)
            zdata_for_reverse.append(output1)
            xdata_back_reverse.append(ui.children[0].value)
            ydata_back_reverse.append(y_value)
            zdata_back_reverse.append(output2)
        else:
            xdata_for.append(ui.children[0].value)
            ydata_for.append(y_value)
            zdata_for.append(output1)
            xdata_back.append(ui.children[0].value)
            ydata_back.append(y_value)
            zdata_back.append(output2)
        
    ax.axes.set_xlim3d(left=0, right=max([10, max(xdata_for, default=0), max(xdata_back, default=0), max(xdata_for_reverse, default=0), max(xdata_back_reverse, default=0)]))
    ax.axes.set_ylim3d(bottom=0, top=max([1, max(ydata_for, default=0), max(ydata_back, default=0), max(ydata_for_reverse, default=0), max(ydata_back_reverse, default=0)]))
    ax.axes.set_zlim3d(bottom=0, top=max([10, max(zdata_for, default=0), max(zdata_back, default=0), max(zdata_for_reverse, default=0), max(zdata_back_reverse, default=0)]))
        
    ax.plot3D(xdata_for_reverse, ydata_for_reverse, zdata_for_reverse, marker='P', markersize=12)
    ax.plot3D(xdata_back_reverse, ydata_back_reverse, zdata_back_reverse, marker='+', markersize=12)
    ax.plot3D(xdata_for, ydata_for, zdata_for, marker='x', markersize=12)
    ax.plot3D(xdata_back, ydata_back, zdata_back, marker='X', markersize=12)
    
    
    plt.show()

button.on_click(on_button_clicked)
reset_graph.on_click(on_reset_clicked)
display(output)

Output()

## Step 7: Delete the Slice

Please delete your slice when you are done with your experiment.

In [152]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")